In [1]:
!pip install -U "huggingface_hub[cli]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 2.1 MB/s eta 0:00:00


In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in 

In [3]:
!pip install requests
!pip install deepl

In [4]:
!pip install transformers==4.40.0 accelerate
!CMAKE_ARGS="-DLLAMA_CUDA=on" pip install llama-cpp-python #양자화 구동을 위한 Llama C++ 설치

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 39.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylin

In [5]:
from llama_cpp import Llama
import random
from transformers import AutoTokenizer
import deepl
import pandas as pd

In [6]:
auth_key = ""  # Replace with your key
translator = deepl.Translator(auth_key)

In [7]:
from huggingface_hub import hf_hub_download
# Hugging Face 레포지토리 ID 및 파일 경로 설정
repo_id = 'QuantFactory/Meta-Llama-3-8B-Instruct-GGUF'  # 예: 'facebook/bart-large'
filename = 'Meta-Llama-3-8B-Instruct.Q4_1.gguf'  # 예: 'model.gguf'

# 특정 파일 다운로드
file_path = hf_hub_download(repo_id=repo_id, filename=filename)

# 다운로드된 파일 경로 출력
print(f"File downloaded to: {file_path}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Meta-Llama-3-8B-Instruct.Q4_1.gguf:   0%|          | 0.00/5.13G [00:00<?, ?B/s]

File downloaded to: /root/.cache/huggingface/hub/models--QuantFactory--Meta-Llama-3-8B-Instruct-GGUF/snapshots/33b3a2a0f06a820b6306ab3aa2020ecb6bcf22da/Meta-Llama-3-8B-Instruct.Q4_1.gguf


In [13]:
import re
import pandas as pd

presentation_list = []

list_2 = []

prompt = '''
당신은 대학 입시의 논술과 면접 학원을 운영하고 있는 선생님입니다. 당신의 주 업무는 제시문으로부터 새로운 질문을 생성하는 것입니다.
당신은 새로운 제시문이 주어지면, 제시문들의 내용을 파악한 후 이와 관련된 질문과 답변을 생성합니다.
또한 질문들은 각 제시문 간의 관계성을 고려해 서로 비교하거나, 연관성을 물어봐야 합니다.
이때, 제시문 [가], 제시문 [나] 등의 단어들이 질문에 등장하고, 이들 간의 관계성 등을 물어봐야 합니다.
제시문의 전반적인 내용에 대해 골고루 질문을 생성하세요. 질문들은 특정 내용만 다루는 것이 아닌, 제시문 전체를 구체적으로 다뤄야 합니다.
'''
format_3 = '''
"Question 1"
"Answer 1"
"Question 2"
"Answer 2"
"Question 3"
"Answer 3"
'''

format_4 = '''
"Question 1"
"Answer 1"
"Question 2"
"Answer 2"
"Question 3"
"Answer 3"
"Question 4"
"Answer 4"
'''

format_5 = '''
"Question 1"
"Answer 1"
"Question 2"
"Answer 2"
"Question 3"
"Answer 3"
"Question 4"
"Answer 4"
"Question 5"
"Answer 5"
'''


In [14]:
for i in range(len(presentation_list)) :
  model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
  tokenizer = AutoTokenizer.from_pretrained(model_id, verbose = 0)
  model = Llama(
      model_path=file_path, #다운로드받은 모델의 위치
      n_ctx=5000,
      n_gpu_layers=-1, verbose = 0        # Number of model layers to offload to GPU
  )
  ###
  presentation = presentation_list[i]
  paragraph = translator.translate_text(presentation, target_lang="EN-US").text
  PROMPT = prompt

  num_of_question = random.randint(3,5)
  if num_of_question == 3 :
    format = format_3
  elif num_of_question == 4 :
    format = format_4
  else :
    format = format_5

  test_text = f'''다음 제시문을 보고, 질문과 답변 세트를 {num_of_question}개 생성하라.

  예를 들어 다음과 같은 제시문 3개가 있다.

  ###
  [제시문 (가)] : 진실을 추구하지만 이야기라는 틀을 벗어날 수 없는 혼종 학문인 역사학은 인문학의 경계에 위치하면서 다른 학문보다 더 어렵기도 하고 더 쉽기도 하다. 역사가들은 원하는 정보 모두를 획득할 때까지 사료를 끊임없이 파헤치고, '사실'을 다루는 자신들의 깊이를 앞세워 여타 학문의 동료들을 괴롭히는 콧대 높은 경험주의자들이다. 이와 동시에 역사책은 흔히 이야기를 중심으로 전개되며, 가장 성공적인 역사서들은 대체로 훌륭한 소설의 속성을 일정하게 갖고 있다.
  역사학의 본질적 혼종성은 과거를 재구성하는 데 있어서 사실성과 허구성 사이의 경계에 관한 논쟁의 핵심적 이유이다.

  [제시문 (나)] : 크리스토퍼 브라우닝(Christopher Browning)은 1942~1943년에 걸쳐 약 38,000명의 유대인 학살 명령을 수행한 독일 101예비경찰대의 재판 기록을 통해 ‘평범한 사람들’이 학살에 가담했던 이유를 설명한다. 유대인을 죽이라는 명령을 받고 당황한 대원들에게 상관은 나이가 좀 더 많은 사람들은 임무를 수행하지 못할 것 같으면 빠져도 좋다고 말했지만, 선택의 가능성에도 불구하고 80~90%의 대원들이 대량 학살에 가담했다. 브라우닝은 사회적 관계로 인해 나약한 인간이 부당한 일을 행할 수 있다고 보았다. 순응주의, 권위에 대한 복종, 임무를 거부할 때 동료들로부터 따돌림을 당할지도 모른다는 두려움이 학살 가담의 결정적 원인이라는 것이다. 브라우닝은 무엇이 보통 사람들을 그토록 잔혹한 범죄에 가담하도록 이끌었는가를 이해하려 했던 것이고 그의 결론은 집단적 순응성의 압도적인 영향이었다.

  [제시문 (다)] : 대니얼 골드하겐(Daniel Goldhagen)은 브라우닝과 동일한 사료를 검토하고 정반대 결론을 내렸다. 그의 결론은 101예비경찰대의 압도적 다수가 동료들의 압력, 복종, 혹은 자신들의 경력 때문에 학살에 가담했던 것이 아니라, 섬뜩할 정도로 냉담하고 잔인한 행동을 묘사한 기록들에서 드러나듯 유대인 학살의 적극적 욕망을 가지고 행동했기 때문이라는 것이다. 골드하겐은, 학살 가담이 내키지 않았고 자신들의 행동을 혐오했다는 대원들의 진술이 자기 변호에 불과하며, 그들은 ‘평범한 보통 사람들’이 아니라 ‘비정상적인 정치문화의 보통 사람들’이라고 보았다. 그의 명제는 단순하고 명확하다. “독일인의 반유대주의적 신념이 홀로코스트를 유발한 핵심 동인이다.” 골드하겐은 사회적 관계에 초점을 맞추기보다는 반유대주의라는 당시 독일 사회의 특수성을 문제시했다. 그의 자명한 주장은 앞선 역사가들과 달랐지만, 상당한 대중적 찬사를 받았다.
  ###

  이 제시문 세트에서 출제된 문제는 다음과 같다.

  1. (가)에서 말한 역사학에서의 허구성을 구체적으로 설명하고, (나)와 (다)에서 발견되는 허구적 요소가 각각 무엇인지 설명하시오.
  2. (가)에서 말한 '혼종성’이 다른 학문 분야에서 어떻게 나타날 수 있는지 예를 들어 설명하시오.

  이처럼 새로운 제시문에 대해 질문을 생성하면 된다.
  이때 생성할 질문들은 예시 질문들과 유사한 형태이어야 한다.
  다음은 예시 질문들이다.

  ["(나), (다), (라) 각각에 근거하여 (가)에 나타난 정부의 물가 변동 파악 방식의 한계점을 설명하시오."]
  ["(가) 또는 (나)에서 문제가 된 상황과 유사한 다른 사례를 제시하고 그 이유를 설명하시오."]
  ["(가)와 (나)에 나타난 독자에 대한 공통된 이해 방식을 (다)의 맥락에서 평가하시오."]
  ["제시문 [다]를 기반으로, [가]와 [나]의 시선 사이의 관계와 그 영향에 대해 논의하시오."]
  ["(가)와 (나)의 밑줄 친 사례에서 관찰되는 문제점의 공통점과 차이점을 구체적으로 설명하시오.]
  ["독자와 저자(혹은 작가)의 관계에 관해 (가)와 (나)에 제시된 입장을 비교하시오."]
  ["(다)의 화자를 만났을 때, (가)와 (나)의 글쓴이가 자신의 입장을 각각 어떻게 변호할지 논하시오."]
  ["제시문 [가], [나], [다]를 종합하여, 현대 사회에서의 인간관계의 핵심 특징을 설명하시오."]

  이처럼 모든 질문은 반드시 "~하시오."처럼 명령 어조이어야 한다. 반드시 이를 지켜야 한다.

  주의점은 다음과 같다.

  ###
  1. 다른 불필요한 문장은 절대로 생성하지 마시오. 오직 질문과 답변만 생성하시오.
  2. 질문은 1~2문장으로, 간결해야 한다.
  3. 각 질문들은 서로 다른 내용으로, 겹치지 않아야 한다.
  4. 모든 질문은 반드시 "~하시오."의 명령조 형식으로 끝나야 한다. 예를 들어 "제시문 [가]와 [나]에 기반하여 [다]에 대한 생각을 설명하시오."와 같은 형탱이어야 한다.
  5. 또한 질문은 최대한 자연스러운 말투이어야 한다.
  6. 지정한 질문과 답변의 세트 개수는, 제시문 전체에 대해서 적용되는 것이다.
  7. 마지막으로 질문에는, 답변에 대한 내용이 절대로 나타나서는 안 된다. 즉, 질문은 구체적인 내용을 물어보는 것이 아닌 포괄적인 질문이어야 한다.
  ###


  또한 생성 형식은
  {format}
  으로 해야 한다.
  반드시 이 생성 형식을 지켜야 한다.


  새로운 제시문은 다음과 같다.

  ###
  {paragraph}
  ###

  이제 질문과 답변 세트를 생성하라. 질문과 답변만 생성하고, 다른 문장은 절대로 생성하지 마시오.
  '''

  prompts = translator.translate_text(PROMPT, target_lang="EN-US").text
  instruction = translator.translate_text(test_text, target_lang="EN-US").text

  messages = [
    {"role": "system", "content": f"{prompts}"},
    {"role": "user", "content": f"{instruction}"}
    ]

  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True, tokenize = False,verbose = 0
  )

  outputs = model(
      input_ids,
      max_tokens = 3000,
      temperature=0.8,
      top_p=0.8,
  )
  generated = outputs['choices'][0]['text'][len(outputs):]
  questions = re.findall(r'Question \d+\n(.*?)\n', generated, re.DOTALL)

  q_list = []
  for i in questions :
    question = translator.translate_text(i, target_lang="KO").text
    if '패션' in question :
      question = question.replace('패션','유행')
    print(question)
    q_list.append(question)

  list_2.append([presentation, num_of_question, PROMPT, test_text, generated, questions, q_list])

columns = ['제시문', '질문 개수', 'prompt', 'instruction', '원본' '생성 질문', '번역']

df_new = pd.DataFrame(list_2, columns=columns)
df_new

ValueError: Shape of passed values is (1, 1), indices imply (1, 6)

In [11]:
test_text

NameError: name 'test_text' is not defined

In [ ]:
presentation1 = '''[문단 (가)] "유행은 누구나 가는 길로 가려는 모방 욕구와 자신을 표현하고 싶은 개성 욕구를 동시에 충족시켜주는 매우 독특한 현상이다. 유행을 따름으로써 자신도 주변 사람들과 똑같이 행동하고 있다는 안도감을 얻으려는 심리, 그리고 다른 사람들과 구별되는 만족감을 얻으려는 심리가 복합적으로 얽혀 있는 것이다. 남을 따르고자 하는 욕구나 소망이 결여되는 경우, 아니면 반대로 개성을 드러내려는 욕구가 결여되는 경우 유행의 영역은 더 이상 존재하지 않게 된다. 예를 들면, 모든 개인이 제각각 특수한 존재가 되려 하고 모방을 기피하는 집단에서는 유행이 발생하지 않는다. 1390년경 피렌체에서 남성 복장의 뚜렷한 유행이 존재하지 않았던 이유는 모두가 각자 독특한 방식으로 차려입고자 했기 때문이다. 반면에 유행이 전체를 지배하게 되면, 즉 처음에는 몇몇 사람이 주도했던 일을 예외 없이 모두가 따라 하게 되면, 그것이 옷이든 음악이든 더 이상 유행이라고 부르지 않는다. 유행은 결코 현재 상태에 머물지 않으며 부단히 진행된다."
[문단 (나)] "옛것이나 선조를 추종하는 경향이 관습의 시대를 지배했다면, 새로운 것을 숭배하고 동시대인들을 모방하려는 경향이 유행의 시대를 특징 짓는다. 이제 사람들은 선조를 닮으려고 하기보다는, 주위 사람들을 닮으려 한다는 것이다. 변화에 대한 애정과 동시대인들에 대한 모방이 유행의 시대를 이끄는 두 가지 중요한 원리이다. 이 원리들에는 선조의 유산을 평가 절하하고 현재의 규범을 중요시하려는 경향이 동반된다. 옛것은 더 이상 존경해야 하는 것으로 여겨지지 않는다. 사람들은 끊임없이 새것을 탐닉하며, 같은 시대에 사는 사람들을 따라 한다. 예를 들면, 근대 유럽의 상류사회는 변화에 대한 열정에 사로잡혀 있었다. 그 열정은 최신 발명품과 이국적 문물에 대한 열망으로 타올랐다. 이탈리아, 스페인, 프랑스가 선도하는 유행을 다른 국가의 상류층 사람들이 앞다퉈 모방하였다. 그들은 새로운 것이면 무엇이든지 뒤쫓아가려고 했고, 가장 최근에 생겨난 변화를 받아들이고자 했다. 중하층 사람들은 다시 상류층의 유행을 모방하였다. 유행은 이런 식으로 사회 전체에 확산하였고, 언제나 새로운 것이 나타날 때마다 그것을 추종하는 상류층에 의해 계속해서 변화하였다.
[문단 (다)] "사람은 사회적인 존재이기 때문에 지위나 역할이 달라지면 자연히 거기에 맞는 행동양식을 하게 된다. 그 행동양식 가운데 하나가 언어 사용이다. 가령,‘엄마, 아빠’와 같은 말은 어린이들의 말이고,‘자네, 댁’과 같은 단어는 어른들의 말이라는 것도 나이에 따른 사회적 행동 양식이 반영된 것이라고 할 수 있다. 이런‘연령 단계’에 의한 언어 차이 외에도, 기성세대와 신세대, 또는 노년층 세대와 청소년층 세대처럼 ‘세대 차이’에 의한 언어 차이도 있을 수 있다. 젊은 세대는 대체로 기성세대와 비교해 볼 때 유행에 민감하고 새로운 변화를 쉽게 받아들인다. 언어에서도 젊은 세대의 이런 특성이 반영된다. 젊은 세대는 ‘반모’(반말 모드), ‘인싸’(무리에 잘 어울려 지내는 사람, 영어 ‘insider’의 의미) 등과 같은 유행어와 신조어를 만드는 주축으로, 대부분의 유행어와 신조어는 자신의 독특한 개성을 표출하려는 젊은 세대를 중심으로 사용된다. 이러한 언어의 유행 현상은 세대 변화에 따른 것이다."
'''

In [ ]:
paragraph = translator.translate_text(presentation1, target_lang="EN-US").text

In [ ]:
PROMPT = '''
당신은 대학 입시의 논술과 면접 학원을 운영하고 있는 선생님입니다. 당신의 주 업무는 제시문으로부터 새로운 질문을 생성하는 것입니다.
당신은 새로운 제시문이 주어지면, 제시문들의 내용을 파악한 후 이와 관련된 질문과 답변을 생성합니다.
또한 질문들은 각 제시문 간의 관계성을 고려해 서로 비교하거나, 연관성을 물어봐야 합니다.
제시문의 전반적인 내용에 대해 골고루 질문을 생성하세요. 질문들은 특정 내용만 다루는 것이 아닌, 제시문 전체를 구체적으로 다뤄야 합니다.

예를 들어 다음과 같은 제시문 3개가 있습니다.

[제시문 (가)] : 진실을 추구하지만 이야기라는 틀을 벗어날 수 없는 혼종 학문인 역사학은 인문학의 경계에 위치하면서 다른 학문보다 더 어렵기도 하고 더 쉽기도 하다. 역사가들은 원하는 정보 모두를 획득할 때까지 사료를 끊임없이 파헤치고, '사실'을 다루는 자신들의 깊이를 앞세워 여타 학문의 동료들을 괴롭히는 콧대 높은 경험주의자들이다. 이와 동시에 역사책은 흔히 이야기를 중심으로 전개되며, 가장 성공적인 역사서들은 대체로 훌륭한 소설의 속성을 일정하게 갖고 있다.
역사학의 본질적 혼종성은 과거를 재구성하는 데 있어서 사실성과 허구성 사이의 경계에 관한 논쟁의 핵심적 이유이다.

[제시문 (나)] : 크리스토퍼 브라우닝(Christopher Browning)은 1942~1943년에 걸쳐 약 38,000명의 유대인 학살 명령을 수행한 독일 101예비경찰대의 재판 기록을 통해 ‘평범한 사람들’이 학살에 가담했던 이유를 설명한다. 유대인을 죽이라는 명령을 받고 당황한 대원들에게 상관은 나이가 좀 더 많은 사람들은 임무를 수행하지 못할 것 같으면 빠져도 좋다고 말했지만, 선택의 가능성에도 불구하고 80~90%의 대원들이 대량 학살에 가담했다. 브라우닝은 사회적 관계로 인해 나약한 인간이 부당한 일을 행할 수 있다고 보았다. 순응주의, 권위에 대한 복종, 임무를 거부할 때 동료들로부터 따돌림을 당할지도 모른다는 두려움이 학살 가담의 결정적 원인이라는 것이다. 브라우닝은 무엇이 보통 사람들을 그토록 잔혹한 범죄에 가담하도록 이끌었는가를 이해하려 했던 것이고 그의 결론은 집단적 순응성의 압도적인 영향이었다.

[제시문 (다)] : 대니얼 골드하겐(Daniel Goldhagen)은 브라우닝과 동일한 사료를 검토하고 정반대 결론을 내렸다. 그의 결론은 101예비경찰대의 압도적 다수가 동료들의 압력, 복종, 혹은 자신들의 경력 때문에 학살에 가담했던 것이 아니라, 섬뜩할 정도로 냉담하고 잔인한 행동을 묘사한 기록들에서 드러나듯 유대인 학살의 적극적 욕망을 가지고 행동했기 때문이라는 것이다. 골드하겐은, 학살 가담이 내키지 않았고 자신들의 행동을 혐오했다는 대원들의 진술이 자기 변호에 불과하며, 그들은 ‘평범한 보통 사람들’이 아니라 ‘비정상적인 정치문화의 보통 사람들’이라고 보았다. 그의 명제는 단순하고 명확하다. “독일인의 반유대주의적 신념이 홀로코스트를 유발한 핵심 동인이다.” 골드하겐은 사회적 관계에 초점을 맞추기보다는 반유대주의라는 당시 독일 사회의 특수성을 문제시했다. 그의 자명한 주장은 앞선 역사가들과 달랐지만, 상당한 대중적 찬사를 받았다.

이 제시문 세트에서 출제된 문제는 다음과 같습니다.

1. (가)에서 말한 역사학에서의 허구성을 구체적으로 설명하고, (나)와 (다)에서 발견되는 허구적 요소가 각각 무엇인지 설명하시오.
2. (가)에서 말한 ‘혼종성’이 다른 학문 분야에서 어떻게 나타날 수 있는지 예를 들어 설명하시오.


이처럼 새로운 제시문이 주어지면 이에 대한 질문을 생성하면 됩니다.
이때 생성할 질문들은 예시 질문들과 유사한 형태이어야 합니다.
다음은 예시 질문들입니다.

["(나), (다), (라) 각각에 근거하여 (가)에 나타난 정부의 물가 변동 파악 방식의 한계점을 설명하시오."]
["(가) 또는 (나)에서 문제가 된 상황과 유사한 다른 사례를 제시하고 그 이유를 설명하시오."]
["(가)와 (나)에 나타난 독자에 대한 공통된 이해 방식을 (다)의 맥락에서 평가하시오."]
["제시문 [다]를 기반으로, [가]와 [나]의 시선 사이의 관계와 그 영향에 대해 논의하시오."]
["(가)와 (나)의 밑줄 친 사례에서 관찰되는 문제점의 공통점과 차이점을 구체적으로 설명하시오.]
["독자와 저자(혹은 작가)의 관계에 관해 (가)와 (나)에 제시된 입장을 비교하시오."]
["(다)의 화자를 만났을 때, (가)와 (나)의 글쓴이가 자신의 입장을 각각 어떻게 변호할지 논하시오."]
["제시문 [가], [나], [다]를 종합하여, 현대 사회에서의 인간관계의 핵심 특징을 설명하시오."]

이처럼 모든 질문은 반드시 "~하시오."처럼 명령 어조이어야 합니다.
'''


In [ ]:
num_of_question = random.randint(3,5)
if num_of_question == 3 :
  format = '''
  "question 1"
  "Answer 1"
  "question 2"
  "Answer 2"
  "question 3"
  "Answer 3"'''
elif num_of_question == 4 :
  format = '''
  "question 1"
  "Answer 1"
  "question 2"
  "Answer 2"
  "question 3"
  "Answer 3"
  "question 4"
  "Answer 4"'''

else :
  format = '''
  "question 1"
  "Answer 1"
  "question 2"
  "Answer 2"
  "question 3"
  "Answer 3"
  "question 4"
  "Answer 4"
  "question 5"
  "Answer 5"'''

In [ ]:
test_text = f'''다음 제시문을 보고, 질문과 답변 세트를 {num_of_question}개 생성하라.

주의점은 다음과 같다.

1. 다른 불필요한 문장은 절대로 생성하지 마시오. 오직 질문과 답변만 생성하시오.
2. 질문은 1~2문장으로, 간결해야 한다.
3. 각 질문들은 서로 다른 내용으로, 겹치지 않아야 한다.
4. 모든 질문은 반드시 "~하시오."의 명령조 형식으로 끝나야 한다. 예를 들어 "제시문 [가]와 [나]에 기반하여 [다]에 대한 생각을 설명하시오."와 같은 형탱이어야 한다.
5. 또한 질문은 최대한 자연스러운 말투이어야 한다.
6. 지정한 질문과 답변의 세트 개수는, 제시문 전체에 대해서 적용되는 것이다.
7. 마지막으로 질문에는, 답변에 대한 내용이 절대로 나타나서는 안 된다. 즉, 질문은 구체적인 내용을 물어보는 것이 아닌 포괄적인 질문이어야 한다.
###

생성 형식은
{format}
으로 해야 한다.
새로운 제시문은 다음과 같다.

###
{paragraph}
###

이제 질문과 답변 세트를 생성하라.
                '''

In [ ]:
prompts = translator.translate_text(PROMPT, target_lang="EN-US").text
instruction = translator.translate_text(test_text, target_lang="EN-US").text

In [ ]:
!huggingface-cli download meta-llama/Meta-Llama-3-8B-Instruct --local-dir='./'

Fetching 17 files: 100% 17/17 [00:00<00:00, 693.29it/s]
/content


#### llama-3 오리지널

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto", n_gpu_layers=-1
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
messages = [
  {"role": "system", "content": f"{prompts}"},
  {"role": "user", "content": f"{instruction}"}
  ]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",verbose = 0
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=2000,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.8,
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Here are five question-answer sets based on the given paragraphs:

**Set 1**
"Describe the relationship between the desire to imitate and the need for individuality in the context of fashion."
The desire to imitate and the need for individuality are two fundamental forces that shape the world of fashion. Fashion is a unique phenomenon that satisfies both the desire to conform and the need to express oneself.

**Set 2**
"How do the principles of the age of fashion differ from those of the age of custom?"
The age of fashion is characterized by a tendency to worship the new and imitate contemporaries, whereas the age of custom is dominated by a tendency to follow the old or forefathers. The love of change and imitation of contemporaries drive the age of fashion, whereas reverence for legacy and tradition define the age of custom.

**Set 3**
"What is the role of social behavior in shaping language, as described in paragraph (c)?"
Social behavior plays a significant role in shaping language

In [ ]:
answer = tokenizer.decode(response, skip_special_tokens=True)
answer

'Here are five question-answer sets based on the given paragraphs:\n\n**Set 1**\n"Describe the relationship between the desire to imitate and the need for individuality in the context of fashion."\nThe desire to imitate and the need for individuality are two fundamental forces that shape the world of fashion. Fashion is a unique phenomenon that satisfies both the desire to conform and the need to express oneself.\n\n**Set 2**\n"How do the principles of the age of fashion differ from those of the age of custom?"\nThe age of fashion is characterized by a tendency to worship the new and imitate contemporaries, whereas the age of custom is dominated by a tendency to follow the old or forefathers. The love of change and imitation of contemporaries drive the age of fashion, whereas reverence for legacy and tradition define the age of custom.\n\n**Set 3**\n"What is the role of social behavior in shaping language, as described in paragraph (c)?"\nSocial behavior plays a significant role in sha

In [ ]:
import re

In [ ]:
questions = re.findall(r'"(.*?)"', answer)

# 결과 출력
for i, question in enumerate(questions, 1):
    print(f"Question {i}: {question}")

Question 1: Describe the relationship between the desire to imitate and the need for individuality in the context of fashion.
Question 2: How do the principles of the age of fashion differ from those of the age of custom?
Question 3: What is the role of social behavior in shaping language, as described in paragraph (c)?
Question 4: How do the concepts of fashion and language fads relate to each other?
Question 5: Explain how the concept of fashion as described in paragraph (a) relates to the idea of individuality.


In [ ]:
questions

['Describe the relationship between the desire to imitate and the need for individuality in the context of fashion.',
 'How do the principles of the age of fashion differ from those of the age of custom?',
 'What is the role of social behavior in shaping language, as described in paragraph (c)?',
 'How do the concepts of fashion and language fads relate to each other?',
 'Explain how the concept of fashion as described in paragraph (a) relates to the idea of individuality.']

In [ ]:
for i in questions :
  question = translator.translate_text(i, target_lang="KO").text
  if '패션' in question :
    question = question.replace('패션','유행')
  print(question)

유행의 맥락에서 모방 욕구와 개성의 필요성 사이의 관계를 설명하세요.
유행의 시대의 원칙은 관습의 시대와 어떻게 다를까요?
(다)항에 설명된 대로 언어를 형성하는 데 있어 사회적 행동의 역할은 무엇인가요?
유행과 언어 유행의 개념은 서로 어떤 관련이 있을까요?
단락 (가)에 설명된 유행의 개념이 개성이라는 개념과 어떤 관련이 있는지 설명하세요.


#### llama-3 양자화

In [22]:
from huggingface_hub import hf_hub_download
# Hugging Face 레포지토리 ID 및 파일 경로 설정
repo_id = 'QuantFactory/Meta-Llama-3-8B-Instruct-GGUF'  # 예: 'facebook/bart-large'
filename = 'Meta-Llama-3-8B-Instruct.Q4_1.gguf'  # 예: 'model.gguf'

# 특정 파일 다운로드
file_path = hf_hub_download(repo_id=repo_id, filename=filename)

# 다운로드된 파일 경로 출력
print(f"File downloaded to: {file_path}")

File downloaded to: /root/.cache/huggingface/hub/models--QuantFactory--Meta-Llama-3-8B-Instruct-GGUF/snapshots/33b3a2a0f06a820b6306ab3aa2020ecb6bcf22da/Meta-Llama-3-8B-Instruct.Q4_1.gguf


In [17]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id, verbose = 0)

model = Llama(
    model_path=file_path, #다운로드받은 모델의 위치
    n_ctx=5000,
    n_gpu_layers=-1, verbose = 0        # Number of model layers to offload to GPU
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [23]:
presentation1 = '''[문단 (가)] "유행은 누구나 가는 길로 가려는 모방 욕구와 자신을 표현하고 싶은 개성 욕구를 동시에 충족시켜주는 매우 독특한 현상이다. 유행을 따름으로써 자신도 주변 사람들과 똑같이 행동하고 있다는 안도감을 얻으려는 심리, 그리고 다른 사람들과 구별되는 만족감을 얻으려는 심리가 복합적으로 얽혀 있는 것이다. 남을 따르고자 하는 욕구나 소망이 결여되는 경우, 아니면 반대로 개성을 드러내려는 욕구가 결여되는 경우 유행의 영역은 더 이상 존재하지 않게 된다. 예를 들면, 모든 개인이 제각각 특수한 존재가 되려 하고 모방을 기피하는 집단에서는 유행이 발생하지 않는다. 1390년경 피렌체에서 남성 복장의 뚜렷한 유행이 존재하지 않았던 이유는 모두가 각자 독특한 방식으로 차려입고자 했기 때문이다. 반면에 유행이 전체를 지배하게 되면, 즉 처음에는 몇몇 사람이 주도했던 일을 예외 없이 모두가 따라 하게 되면, 그것이 옷이든 음악이든 더 이상 유행이라고 부르지 않는다. 유행은 결코 현재 상태에 머물지 않으며 부단히 진행된다."
[문단 (나)] "옛것이나 선조를 추종하는 경향이 관습의 시대를 지배했다면, 새로운 것을 숭배하고 동시대인들을 모방하려는 경향이 유행의 시대를 특징 짓는다. 이제 사람들은 선조를 닮으려고 하기보다는, 주위 사람들을 닮으려 한다는 것이다. 변화에 대한 애정과 동시대인들에 대한 모방이 유행의 시대를 이끄는 두 가지 중요한 원리이다. 이 원리들에는 선조의 유산을 평가 절하하고 현재의 규범을 중요시하려는 경향이 동반된다. 옛것은 더 이상 존경해야 하는 것으로 여겨지지 않는다. 사람들은 끊임없이 새것을 탐닉하며, 같은 시대에 사는 사람들을 따라 한다. 예를 들면, 근대 유럽의 상류사회는 변화에 대한 열정에 사로잡혀 있었다. 그 열정은 최신 발명품과 이국적 문물에 대한 열망으로 타올랐다. 이탈리아, 스페인, 프랑스가 선도하는 유행을 다른 국가의 상류층 사람들이 앞다퉈 모방하였다. 그들은 새로운 것이면 무엇이든지 뒤쫓아가려고 했고, 가장 최근에 생겨난 변화를 받아들이고자 했다. 중하층 사람들은 다시 상류층의 유행을 모방하였다. 유행은 이런 식으로 사회 전체에 확산하였고, 언제나 새로운 것이 나타날 때마다 그것을 추종하는 상류층에 의해 계속해서 변화하였다.
[문단 (다)] "사람은 사회적인 존재이기 때문에 지위나 역할이 달라지면 자연히 거기에 맞는 행동양식을 하게 된다. 그 행동양식 가운데 하나가 언어 사용이다. 가령,‘엄마, 아빠’와 같은 말은 어린이들의 말이고,‘자네, 댁’과 같은 단어는 어른들의 말이라는 것도 나이에 따른 사회적 행동 양식이 반영된 것이라고 할 수 있다. 이런‘연령 단계’에 의한 언어 차이 외에도, 기성세대와 신세대, 또는 노년층 세대와 청소년층 세대처럼 ‘세대 차이’에 의한 언어 차이도 있을 수 있다. 젊은 세대는 대체로 기성세대와 비교해 볼 때 유행에 민감하고 새로운 변화를 쉽게 받아들인다. 언어에서도 젊은 세대의 이런 특성이 반영된다. 젊은 세대는 ‘반모’(반말 모드), ‘인싸’(무리에 잘 어울려 지내는 사람, 영어 ‘insider’의 의미) 등과 같은 유행어와 신조어를 만드는 주축으로, 대부분의 유행어와 신조어는 자신의 독특한 개성을 표출하려는 젊은 세대를 중심으로 사용된다. 이러한 언어의 유행 현상은 세대 변화에 따른 것이다."
'''

In [24]:
paragraph = translator.translate_text(presentation1, target_lang="EN-US").text

In [40]:
PROMPT = '''
당신은 대학 입시의 논술과 면접 학원을 운영하고 있는 선생님입니다. 당신의 주 업무는 제시문으로부터 새로운 질문을 생성하는 것입니다.
당신은 새로운 제시문이 주어지면, 제시문들의 내용을 파악한 후 이와 관련된 질문과 답변을 생성합니다.
또한 질문들은 각 제시문 간의 관계성을 고려해 서로 비교하거나, 연관성을 물어봐야 합니다.
이때, 제시문 [가], 제시문 [나] 등의 단어들이 질문에 등장하고, 이들 간의 관계성 등을 물어봐야 합니다.
제시문의 전반적인 내용에 대해 골고루 질문을 생성하세요. 질문들은 특정 내용만 다루는 것이 아닌, 제시문 전체를 구체적으로 다뤄야 합니다.
'''


In [50]:
num_of_question = random.randint(3,5)
if num_of_question == 3 :
  format = '''
  "Question 1"
  "Answer 1"
  "Question 2"
  "Answer 2"
  "Question 3"
  "Answer 3"'''
elif num_of_question == 4 :
  format = '''
  "Question 1"
  "Answer 1"
  "Question 2"
  "Answer 2"
  "Question 3"
  "Answer 3"
  "Question 4"
  "Answer 4"'''

else :
  format = '''
  "Question 1"
  "Answer 1"
  "Question 2"
  "Answer 2"
  "Question 3"
  "Answer 3"
  "Question 4"
  "Answer 4"
  "Question 5"
  "Answer 5"'''

In [51]:
test_text = f'''다음 제시문을 보고, 질문과 답변 세트를 {num_of_question}개 생성하라.

예를 들어 다음과 같은 제시문 3개가 있다.
###
[제시문 (가)] : 진실을 추구하지만 이야기라는 틀을 벗어날 수 없는 혼종 학문인 역사학은 인문학의 경계에 위치하면서 다른 학문보다 더 어렵기도 하고 더 쉽기도 하다. 역사가들은 원하는 정보 모두를 획득할 때까지 사료를 끊임없이 파헤치고, '사실'을 다루는 자신들의 깊이를 앞세워 여타 학문의 동료들을 괴롭히는 콧대 높은 경험주의자들이다. 이와 동시에 역사책은 흔히 이야기를 중심으로 전개되며, 가장 성공적인 역사서들은 대체로 훌륭한 소설의 속성을 일정하게 갖고 있다.
역사학의 본질적 혼종성은 과거를 재구성하는 데 있어서 사실성과 허구성 사이의 경계에 관한 논쟁의 핵심적 이유이다.

[제시문 (나)] : 크리스토퍼 브라우닝(Christopher Browning)은 1942~1943년에 걸쳐 약 38,000명의 유대인 학살 명령을 수행한 독일 101예비경찰대의 재판 기록을 통해 ‘평범한 사람들’이 학살에 가담했던 이유를 설명한다. 유대인을 죽이라는 명령을 받고 당황한 대원들에게 상관은 나이가 좀 더 많은 사람들은 임무를 수행하지 못할 것 같으면 빠져도 좋다고 말했지만, 선택의 가능성에도 불구하고 80~90%의 대원들이 대량 학살에 가담했다. 브라우닝은 사회적 관계로 인해 나약한 인간이 부당한 일을 행할 수 있다고 보았다. 순응주의, 권위에 대한 복종, 임무를 거부할 때 동료들로부터 따돌림을 당할지도 모른다는 두려움이 학살 가담의 결정적 원인이라는 것이다. 브라우닝은 무엇이 보통 사람들을 그토록 잔혹한 범죄에 가담하도록 이끌었는가를 이해하려 했던 것이고 그의 결론은 집단적 순응성의 압도적인 영향이었다.

[제시문 (다)] : 대니얼 골드하겐(Daniel Goldhagen)은 브라우닝과 동일한 사료를 검토하고 정반대 결론을 내렸다. 그의 결론은 101예비경찰대의 압도적 다수가 동료들의 압력, 복종, 혹은 자신들의 경력 때문에 학살에 가담했던 것이 아니라, 섬뜩할 정도로 냉담하고 잔인한 행동을 묘사한 기록들에서 드러나듯 유대인 학살의 적극적 욕망을 가지고 행동했기 때문이라는 것이다. 골드하겐은, 학살 가담이 내키지 않았고 자신들의 행동을 혐오했다는 대원들의 진술이 자기 변호에 불과하며, 그들은 ‘평범한 보통 사람들’이 아니라 ‘비정상적인 정치문화의 보통 사람들’이라고 보았다. 그의 명제는 단순하고 명확하다. “독일인의 반유대주의적 신념이 홀로코스트를 유발한 핵심 동인이다.” 골드하겐은 사회적 관계에 초점을 맞추기보다는 반유대주의라는 당시 독일 사회의 특수성을 문제시했다. 그의 자명한 주장은 앞선 역사가들과 달랐지만, 상당한 대중적 찬사를 받았다.
###
이 제시문 세트에서 출제된 문제는 다음과 같다.

1. (가)에서 말한 역사학에서의 허구성을 구체적으로 설명하고, (나)와 (다)에서 발견되는 허구적 요소가 각각 무엇인지 설명하시오.
2. (가)에서 말한 '혼종성’이 다른 학문 분야에서 어떻게 나타날 수 있는지 예를 들어 설명하시오.

이처럼 새로운 제시문에 대해 질문을 생성하면 된다.
이때 생성할 질문들은 예시 질문들과 유사한 형태이어야 한다.
다음은 예시 질문들이다.

["(나), (다), (라) 각각에 근거하여 (가)에 나타난 정부의 물가 변동 파악 방식의 한계점을 설명하시오."]
["(가) 또는 (나)에서 문제가 된 상황과 유사한 다른 사례를 제시하고 그 이유를 설명하시오."]
["(가)와 (나)에 나타난 독자에 대한 공통된 이해 방식을 (다)의 맥락에서 평가하시오."]
["제시문 [다]를 기반으로, [가]와 [나]의 시선 사이의 관계와 그 영향에 대해 논의하시오."]
["(가)와 (나)의 밑줄 친 사례에서 관찰되는 문제점의 공통점과 차이점을 구체적으로 설명하시오.]
["독자와 저자(혹은 작가)의 관계에 관해 (가)와 (나)에 제시된 입장을 비교하시오."]
["(다)의 화자를 만났을 때, (가)와 (나)의 글쓴이가 자신의 입장을 각각 어떻게 변호할지 논하시오."]
["제시문 [가], [나], [다]를 종합하여, 현대 사회에서의 인간관계의 핵심 특징을 설명하시오."]

이처럼 모든 질문은 반드시 "~하시오."처럼 명령 어조이어야 한다. 반드시 이를 지켜야 한다.

주의점은 다음과 같다.

###
1. 다른 불필요한 문장은 절대로 생성하지 마시오. 오직 질문과 답변만 생성하시오.
2. 질문은 1~2문장으로, 간결해야 한다.
3. 각 질문들은 서로 다른 내용으로, 겹치지 않아야 한다.
4. 모든 질문은 반드시 "~하시오."의 명령조 형식으로 끝나야 한다. 예를 들어 "제시문 [가]와 [나]에 기반하여 [다]에 대한 생각을 설명하시오."와 같은 형탱이어야 한다.
5. 또한 질문은 최대한 자연스러운 말투이어야 한다.
6. 지정한 질문과 답변의 세트 개수는, 제시문 전체에 대해서 적용되는 것이다.
7. 마지막으로 질문에는, 답변에 대한 내용이 절대로 나타나서는 안 된다. 즉, 질문은 구체적인 내용을 물어보는 것이 아닌 포괄적인 질문이어야 한다.
###


또한 생성 형식은
{format}
으로 해야 한다.
반드시 이 생성 형식을 지켜야 한다.


새로운 제시문은 다음과 같다.

###
{paragraph}
###

이제 질문과 답변 세트를 생성하라. 질문과 답변만 생성하고, 다른 문장은 절대로 생성하지 마시오.
                '''

In [52]:
prompts = translator.translate_text(PROMPT, target_lang="EN-US").text
instruction = translator.translate_text(test_text, target_lang="EN-US").text

In [53]:
messages = [
  {"role": "system", "content": f"{prompts}"},
  {"role": "user", "content": f"{instruction}"}
  ]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True, tokenize = False,verbose = 0
)

outputs = model(
    input_ids,
    max_tokens = 3000,
    temperature=0.7,
    top_p=0.8,
)

generated = outputs['choices'][0]['text'][len(outputs):]
print(generated)

s the generated set of questions and answers:

Question 1
Please describe how the concept of fashion relates to both individuality and conformity.

Answer 1
Fashion satisfies both desires, combining the need to feel reassured that you're doing the same thing as everyone else with the satisfaction of being different from everyone else.

Question 2
What is the significance of the trend-setting behavior in Florence around 1390?

Answer 2
In this era, there was no clear trend in men's clothing because everyone wanted to dress in their own unique way, indicating that fashion arises when people are driven by a desire to conform and be different.

Question 3
How do the principles of imitation and change drive the age of fashion?

Answer 3
The love of change and the imitation of contemporaries drive the age of fashion, characterized by a tendency to devalue legacy and value current norms, resulting in a constant pursuit of new things and trends.

Question 4
Please explain how social behavior i

In [55]:
questions = re.findall(r'Question \d+\n(.*?)\n', generated, re.DOTALL)

# 결과 출력
for i, question in enumerate(questions, 1):
    print(f"Question {i}: {question.strip()}")

Question 1: Please describe how the concept of fashion relates to both individuality and conformity.
Question 2: What is the significance of the trend-setting behavior in Florence around 1390?
Question 3: How do the principles of imitation and change drive the age of fashion?
Question 4: Please explain how social behavior influences language use.
Question 5: What is the relationship between language fads and generational differences?


In [58]:
q_list = []
for i in questions :
  question = translator.translate_text(i, target_lang="KO").text
  if '패션' in question :
    question = question.replace('패션','유행')
  print(question)
  q_list.append(question)

유행의 개념이 개성과 적합성 모두와 어떤 관련이 있는지 설명해 주세요.
1390년경 피렌체에서 유행을 선도하는 행동의 의미는 무엇인가요?
모방과 변화의 원리가 유행의 시대를 어떻게 이끌고 있을까요?
사회적 행동이 언어 사용에 어떤 영향을 미치는지 설명해 주세요.
언어 유행과 세대 차이의 관계는 무엇인가요?


In [59]:
import re
import pandas as pd

presentation_list = []

list_2 = []

prompt = '''
당신은 대학 입시의 논술과 면접 학원을 운영하고 있는 선생님입니다. 당신의 주 업무는 제시문으로부터 새로운 질문을 생성하는 것입니다.
당신은 새로운 제시문이 주어지면, 제시문들의 내용을 파악한 후 이와 관련된 질문과 답변을 생성합니다.
또한 질문들은 각 제시문 간의 관계성을 고려해 서로 비교하거나, 연관성을 물어봐야 합니다.
이때, 제시문 [가], 제시문 [나] 등의 단어들이 질문에 등장하고, 이들 간의 관계성 등을 물어봐야 합니다.
제시문의 전반적인 내용에 대해 골고루 질문을 생성하세요. 질문들은 특정 내용만 다루는 것이 아닌, 제시문 전체를 구체적으로 다뤄야 합니다.
'''
format_3 = '''
"Question 1"
"Answer 1"
"Question 2"
"Answer 2"
"Question 3"
"Answer 3"
'''

format_4 = '''
"Question 1"
"Answer 1"
"Question 2"
"Answer 2"
"Question 3"
"Answer 3"
"Question 4"
"Answer 4"
'''

format_5 = '''
"Question 1"
"Answer 1"
"Question 2"
"Answer 2"
"Question 3"
"Answer 3"
"Question 4"
"Answer 4"
"Question 5"
"Answer 5"
'''


In [60]:
for i in range(len(presentation_list)) :
  model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
  tokenizer = AutoTokenizer.from_pretrained(model_id, verbose = 0)
  model = Llama(
      model_path=file_path, #다운로드받은 모델의 위치
      n_ctx=5000,
      n_gpu_layers=-1, verbose = 0        # Number of model layers to offload to GPU
  )
  ###
  presentation = presentation_list[i]
  paragraph = translator.translate_text(presentation, target_lang="EN-US").text
  PROMPT = prompt

  num_of_question = random.randint(3,5)
  if num_of_question == 3 :
    format = format_3
  elif num_of_question == 4 :
    format = format_4
  else :
    format = format_5

  test_text = f'''다음 제시문을 보고, 질문과 답변 세트를 {num_of_question}개 생성하라.

  예를 들어 다음과 같은 제시문 3개가 있다.

  ###
  [제시문 (가)] : 진실을 추구하지만 이야기라는 틀을 벗어날 수 없는 혼종 학문인 역사학은 인문학의 경계에 위치하면서 다른 학문보다 더 어렵기도 하고 더 쉽기도 하다. 역사가들은 원하는 정보 모두를 획득할 때까지 사료를 끊임없이 파헤치고, '사실'을 다루는 자신들의 깊이를 앞세워 여타 학문의 동료들을 괴롭히는 콧대 높은 경험주의자들이다. 이와 동시에 역사책은 흔히 이야기를 중심으로 전개되며, 가장 성공적인 역사서들은 대체로 훌륭한 소설의 속성을 일정하게 갖고 있다.
  역사학의 본질적 혼종성은 과거를 재구성하는 데 있어서 사실성과 허구성 사이의 경계에 관한 논쟁의 핵심적 이유이다.

  [제시문 (나)] : 크리스토퍼 브라우닝(Christopher Browning)은 1942~1943년에 걸쳐 약 38,000명의 유대인 학살 명령을 수행한 독일 101예비경찰대의 재판 기록을 통해 ‘평범한 사람들’이 학살에 가담했던 이유를 설명한다. 유대인을 죽이라는 명령을 받고 당황한 대원들에게 상관은 나이가 좀 더 많은 사람들은 임무를 수행하지 못할 것 같으면 빠져도 좋다고 말했지만, 선택의 가능성에도 불구하고 80~90%의 대원들이 대량 학살에 가담했다. 브라우닝은 사회적 관계로 인해 나약한 인간이 부당한 일을 행할 수 있다고 보았다. 순응주의, 권위에 대한 복종, 임무를 거부할 때 동료들로부터 따돌림을 당할지도 모른다는 두려움이 학살 가담의 결정적 원인이라는 것이다. 브라우닝은 무엇이 보통 사람들을 그토록 잔혹한 범죄에 가담하도록 이끌었는가를 이해하려 했던 것이고 그의 결론은 집단적 순응성의 압도적인 영향이었다.

  [제시문 (다)] : 대니얼 골드하겐(Daniel Goldhagen)은 브라우닝과 동일한 사료를 검토하고 정반대 결론을 내렸다. 그의 결론은 101예비경찰대의 압도적 다수가 동료들의 압력, 복종, 혹은 자신들의 경력 때문에 학살에 가담했던 것이 아니라, 섬뜩할 정도로 냉담하고 잔인한 행동을 묘사한 기록들에서 드러나듯 유대인 학살의 적극적 욕망을 가지고 행동했기 때문이라는 것이다. 골드하겐은, 학살 가담이 내키지 않았고 자신들의 행동을 혐오했다는 대원들의 진술이 자기 변호에 불과하며, 그들은 ‘평범한 보통 사람들’이 아니라 ‘비정상적인 정치문화의 보통 사람들’이라고 보았다. 그의 명제는 단순하고 명확하다. “독일인의 반유대주의적 신념이 홀로코스트를 유발한 핵심 동인이다.” 골드하겐은 사회적 관계에 초점을 맞추기보다는 반유대주의라는 당시 독일 사회의 특수성을 문제시했다. 그의 자명한 주장은 앞선 역사가들과 달랐지만, 상당한 대중적 찬사를 받았다.
  ###

  이 제시문 세트에서 출제된 문제는 다음과 같다.

  1. (가)에서 말한 역사학에서의 허구성을 구체적으로 설명하고, (나)와 (다)에서 발견되는 허구적 요소가 각각 무엇인지 설명하시오.
  2. (가)에서 말한 '혼종성’이 다른 학문 분야에서 어떻게 나타날 수 있는지 예를 들어 설명하시오.

  이처럼 새로운 제시문에 대해 질문을 생성하면 된다.
  이때 생성할 질문들은 예시 질문들과 유사한 형태이어야 한다.
  다음은 예시 질문들이다.

  ["(나), (다), (라) 각각에 근거하여 (가)에 나타난 정부의 물가 변동 파악 방식의 한계점을 설명하시오."]
  ["(가) 또는 (나)에서 문제가 된 상황과 유사한 다른 사례를 제시하고 그 이유를 설명하시오."]
  ["(가)와 (나)에 나타난 독자에 대한 공통된 이해 방식을 (다)의 맥락에서 평가하시오."]
  ["제시문 [다]를 기반으로, [가]와 [나]의 시선 사이의 관계와 그 영향에 대해 논의하시오."]
  ["(가)와 (나)의 밑줄 친 사례에서 관찰되는 문제점의 공통점과 차이점을 구체적으로 설명하시오.]
  ["독자와 저자(혹은 작가)의 관계에 관해 (가)와 (나)에 제시된 입장을 비교하시오."]
  ["(다)의 화자를 만났을 때, (가)와 (나)의 글쓴이가 자신의 입장을 각각 어떻게 변호할지 논하시오."]
  ["제시문 [가], [나], [다]를 종합하여, 현대 사회에서의 인간관계의 핵심 특징을 설명하시오."]

  이처럼 모든 질문은 반드시 "~하시오."처럼 명령 어조이어야 한다. 반드시 이를 지켜야 한다.

  주의점은 다음과 같다.

  ###
  1. 다른 불필요한 문장은 절대로 생성하지 마시오. 오직 질문과 답변만 생성하시오.
  2. 질문은 1~2문장으로, 간결해야 한다.
  3. 각 질문들은 서로 다른 내용으로, 겹치지 않아야 한다.
  4. 모든 질문은 반드시 "~하시오."의 명령조 형식으로 끝나야 한다. 예를 들어 "제시문 [가]와 [나]에 기반하여 [다]에 대한 생각을 설명하시오."와 같은 형탱이어야 한다.
  5. 또한 질문은 최대한 자연스러운 말투이어야 한다.
  6. 지정한 질문과 답변의 세트 개수는, 제시문 전체에 대해서 적용되는 것이다.
  7. 마지막으로 질문에는, 답변에 대한 내용이 절대로 나타나서는 안 된다. 즉, 질문은 구체적인 내용을 물어보는 것이 아닌 포괄적인 질문이어야 한다.
  ###


  또한 생성 형식은
  {format}
  으로 해야 한다.
  반드시 이 생성 형식을 지켜야 한다.


  새로운 제시문은 다음과 같다.

  ###
  {paragraph}
  ###

  이제 질문과 답변 세트를 생성하라. 질문과 답변만 생성하고, 다른 문장은 절대로 생성하지 마시오.
  '''

  prompts = translator.translate_text(PROMPT, target_lang="EN-US").text
  instruction = translator.translate_text(test_text, target_lang="EN-US").text

  messages = [
    {"role": "system", "content": f"{prompts}"},
    {"role": "user", "content": f"{instruction}"}
    ]

  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True, tokenize = False,verbose = 0
  )

  outputs = model(
      input_ids,
      max_tokens = 3000,
      temperature=0.8,
      top_p=0.8,
  )
  generated = outputs['choices'][0]['text'][len(outputs):]
  questions = re.findall(r'Question \d+\n(.*?)\n', generated, re.DOTALL)

  q_list = []
  for i in questions :
    question = translator.translate_text(i, target_lang="KO").text
    if '패션' in question :
      question = question.replace('패션','유행')
    print(question)
    q_list.append(question)

  list_2.append([presentation, num_of_question, PROMPT, test_text, generated, questions, q_list])

columns = ['제시문', '질문 개수', 'prompt', 'instruction', '원본' '생성 질문', '번역']

df_new = pd.DataFrame(list_2, columns=columns)
df_new

NameError: name 'pd' is not defined

#### 유사도 비교

In [ ]:
!pip install transformers torch sentencepiece


In [ ]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

# KoBERT 모델 및 토크나이저 불러오기
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertModel.from_pretrained('monologg/kobert')

# 두 문장 정의

for i in q_list :
  sentence1 = i
  sentence2 = "제시문 (가), (나), (다)에는 유행에 대한 다양한 관점이 포함되어 있다. 그 관점을 비교, 분석하시오."

  # 문장 토크나이즈 및 인코딩
  inputs = tokenizer([sentence1, sentence2], return_tensors='pt', padding=True, truncation=True)

  # BERT 모델을 사용하여 임베딩 벡터 생성
  with torch.no_grad():
      outputs = model(**inputs)
      embeddings = outputs.last_hidden_state.mean(dim=1)

  # 코사인 유사도 계산
  similarity = cosine_similarity(embeddings[0].unsqueeze(0), embeddings[1].unsqueeze(0))

  # 결과 출력
  print(f"The cosine similarity between the sentences is: {similarity[0][0]}")

tokenizer_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

The cosine similarity between the sentences is: 0.643508791923523
The cosine similarity between the sentences is: 0.6531752943992615
The cosine similarity between the sentences is: 0.7031717300415039
The cosine similarity between the sentences is: 0.728096604347229
The cosine similarity between the sentences is: 0.7003342509269714


In [ ]:
q_list2= ['유행의 맥락에서 모방 욕구와 개성의 필요성 사이의 관계를 설명하세요.', '유행의 시대의 원칙은 관습의 시대와 어떻게 다를까요?', '(다)항에 설명된 대로 언어를 형성하는 데 있어 사회적 행동의 역할은 무엇인가요?','유행과 언어 유행의 개념은 서로 어떤 관련이 있을까요?','단락 (가)에 설명된 유행의 개념이 개성이라는 개념과 어떤 관련이 있는지 설명하세요.']
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

# KoBERT 모델 및 토크나이저 불러오기
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model_kobert = BertModel.from_pretrained('monologg/kobert')

# 두 문장 정의

for i in q_list2 :
  sentence1 = i
  sentence2 = "제시문 (가), (나), (다)에는 유행에 대한 다양한 관점이 포함되어 있다. 그 관점을 비교, 분석하시오."

  # 문장 토크나이즈 및 인코딩
  inputs = tokenizer([sentence1, sentence2], return_tensors='pt', padding=True, truncation=True)

  # BERT 모델을 사용하여 임베딩 벡터 생성
  with torch.no_grad():
      outputs = model_kobert(**inputs)
      embeddings = outputs.last_hidden_state.mean(dim=1)

  # 코사인 유사도 계산
  similarity = cosine_similarity(embeddings[0].unsqueeze(0), embeddings[1].unsqueeze(0))

  # 결과 출력
  print(f"The cosine similarity between the sentences is: {similarity[0][0]}")

The cosine similarity between the sentences is: 0.6847736835479736
The cosine similarity between the sentences is: 0.6395291090011597
The cosine similarity between the sentences is: 0.7937606573104858
The cosine similarity between the sentences is: 0.6531752943992615
The cosine similarity between the sentences is: 0.8139324188232422


#### 새로운 제시문

In [ ]:
presentation2 = '''[문단 (가)] " 공공미술은 공공장소에 설치되므로 미술관에 전시된 작품과 달리 원하지 않는 시민들에게도 노출되기 마련이다. 따라서 공공미술을 기획할 때는 대중의 미적 만족을 고려해야 한다. 일반적 취향을 벗어나 아름답기는커녕 불쾌감만 주는 작품에 공공 재정을 지출하는 것은 정당하지 않다."
[문단 (나)] "공공미술의 공공성은 그 목적에서 찾아야 한다. 누구의 심기도 건드리지 않기 위해 무난하고 의례적인 작품만 선정한다거나, 작품의 선택을 주민투표에 맡긴다면 예술을 지원할 이유가 없다. 공공미술은 대중의 취향을 교육하고 시민에게 더 나은 삶의 가치를 전달할 기회다. 어떤 작품이 그럴 만한 것인지 판단할 수 있는 사람은 인정된 전문가들이다."
[문단 (다)] "1981년 리차드 세라는 정부의 지원을 받아 길이 36미터의 녹슨 강철판인 <기울어진 호>를 맨해튼의 작은 광장에 설치했다. 시민의 동선을 변경하여 광장의 기능을 다시 생각하게 하려는 것이 작가의 의도였다. 이 작품은 전통적인 조각처럼 관조의 대상에 머무르지 않는다. 작품에 반응하는 관람객의 행동과 이로 인해 새롭게 규정되는 공간까지 작품의 일부가 되는 것이다. 하지만 관습에 익숙한 눈에 작품은 건축 폐기물에 불과했다. 충격과 불편함에 시민들의 항의가 이어졌고 논란 끝에 작품은 89년에 철거되었다. 1982년 유사한 양식의 조형물이 워싱턴 국회의사당 인근에 세워졌다. 공모전에서 마야 린의 <베트남 참전 용사 기념물>이 선정된 것이다. 길이 150미터의 검은 화강암 벽은 중간이 한 번 꺾여 있을 뿐 단순했다. 작가는 기존의 전쟁기념물과 달리 전쟁이 아닌 사람을 기리겠다고 생각하여, 베트남에서 죽은 이들의 이름을 사망 연도순으로 벽에 새겨 넣었다. 전혀 영웅적이지 않다는 일부의 불만에도 불구하고, 이 조형물은 몇 년 만에 워싱턴에서 가장 많은 사람이 방문하는 장소가 되었다. 베트남 전쟁으로 양분된 미국은 오랫동안 정치적이고 이념적인 대립을 겪었고, 그 상처는 깊었다. 작품에서 린은 이 논쟁적인 사안에 화해나 종결을 제안하지 않았다. 참전 용사부터 반전 평화주의자까지, 입장이 다른 모든 관람객이 이곳에서원하는 방식으로 전쟁을 반추할 수 있었다."
'''

In [ ]:
paragraph2 = translator.translate_text(presentation2, target_lang="EN-US").text

In [ ]:
PROMPT2 = '''
당신은 대학 입시의 논술과 면접 학원을 운영하고 있는 선생님입니다. 당신의 주 업무는 제시문으로부터 새로운 질문을 생성하는 것입니다.
당신은 새로운 제시문이 주어지면, 제시문들의 내용을 파악한 후 이와 관련된 질문과 답변을 생성합니다.
또한 질문들은 각 제시문 간의 관계성을 고려해 서로 비교하거나, 연관성을 물어봐야 합니다.
제시문의 전반적인 내용에 대해 골고루 질문을 생성하세요. 질문들은 특정 내용만 다루는 것이 아닌, 제시문 전체를 구체적으로 다뤄야 합니다.

예를 들어 다음과 같은 제시문 3개가 있습니다.

[제시문 (가)] : 진실을 추구하지만 이야기라는 틀을 벗어날 수 없는 혼종 학문인 역사학은 인문학의 경계에 위치하면서 다른 학문보다 더 어렵기도 하고 더 쉽기도 하다. 역사가들은 원하는 정보 모두를 획득할 때까지 사료를 끊임없이 파헤치고, '사실'을 다루는 자신들의 깊이를 앞세워 여타 학문의 동료들을 괴롭히는 콧대 높은 경험주의자들이다. 이와 동시에 역사책은 흔히 이야기를 중심으로 전개되며, 가장 성공적인 역사서들은 대체로 훌륭한 소설의 속성을 일정하게 갖고 있다.
역사학의 본질적 혼종성은 과거를 재구성하는 데 있어서 사실성과 허구성 사이의 경계에 관한 논쟁의 핵심적 이유이다.

[제시문 (나)] : 크리스토퍼 브라우닝(Christopher Browning)은 1942~1943년에 걸쳐 약 38,000명의 유대인 학살 명령을 수행한 독일 101예비경찰대의 재판 기록을 통해 ‘평범한 사람들’이 학살에 가담했던 이유를 설명한다. 유대인을 죽이라는 명령을 받고 당황한 대원들에게 상관은 나이가 좀 더 많은 사람들은 임무를 수행하지 못할 것 같으면 빠져도 좋다고 말했지만, 선택의 가능성에도 불구하고 80~90%의 대원들이 대량 학살에 가담했다. 브라우닝은 사회적 관계로 인해 나약한 인간이 부당한 일을 행할 수 있다고 보았다. 순응주의, 권위에 대한 복종, 임무를 거부할 때 동료들로부터 따돌림을 당할지도 모른다는 두려움이 학살 가담의 결정적 원인이라는 것이다. 브라우닝은 무엇이 보통 사람들을 그토록 잔혹한 범죄에 가담하도록 이끌었는가를 이해하려 했던 것이고 그의 결론은 집단적 순응성의 압도적인 영향이었다.

[제시문 (다)] : 대니얼 골드하겐(Daniel Goldhagen)은 브라우닝과 동일한 사료를 검토하고 정반대 결론을 내렸다. 그의 결론은 101예비경찰대의 압도적 다수가 동료들의 압력, 복종, 혹은 자신들의 경력 때문에 학살에 가담했던 것이 아니라, 섬뜩할 정도로 냉담하고 잔인한 행동을 묘사한 기록들에서 드러나듯 유대인 학살의 적극적 욕망을 가지고 행동했기 때문이라는 것이다. 골드하겐은, 학살 가담이 내키지 않았고 자신들의 행동을 혐오했다는 대원들의 진술이 자기 변호에 불과하며, 그들은 ‘평범한 보통 사람들’이 아니라 ‘비정상적인 정치문화의 보통 사람들’이라고 보았다. 그의 명제는 단순하고 명확하다. “독일인의 반유대주의적 신념이 홀로코스트를 유발한 핵심 동인이다.” 골드하겐은 사회적 관계에 초점을 맞추기보다는 반유대주의라는 당시 독일 사회의 특수성을 문제시했다. 그의 자명한 주장은 앞선 역사가들과 달랐지만, 상당한 대중적 찬사를 받았다.

이 제시문 세트에서 출제된 문제는 다음과 같습니다.

1. (가)에서 말한 역사학에서의 허구성을 구체적으로 설명하고, (나)와 (다)에서 발견되는 허구적 요소가 각각 무엇인지 설명하시오.
2. (가)에서 말한 ‘혼종성’이 다른 학문 분야에서 어떻게 나타날 수 있는지 예를 들어 설명하시오.


이처럼 새로운 제시문이 주어지면 이에 대한 질문을 생성하면 됩니다.
이때 생성할 질문들은 예시 질문들과 유사한 형태이어야 합니다.
다음은 예시 질문들입니다.

["(나), (다), (라) 각각에 근거하여 (가)에 나타난 정부의 물가 변동 파악 방식의 한계점을 설명하시오."]
["(가) 또는 (나)에서 문제가 된 상황과 유사한 다른 사례를 제시하고 그 이유를 설명하시오."]
["(가)와 (나)에 나타난 독자에 대한 공통된 이해 방식을 (다)의 맥락에서 평가하시오."]
["제시문 [다]를 기반으로, [가]와 [나]의 시선 사이의 관계와 그 영향에 대해 논의하시오."]
["(가)와 (나)의 밑줄 친 사례에서 관찰되는 문제점의 공통점과 차이점을 구체적으로 설명하시오.]
["독자와 저자(혹은 작가)의 관계에 관해 (가)와 (나)에 제시된 입장을 비교하시오."]
["(다)의 화자를 만났을 때, (가)와 (나)의 글쓴이가 자신의 입장을 각각 어떻게 변호할지 논하시오."]
["제시문 [가], [나], [다]를 종합하여, 현대 사회에서의 인간관계의 핵심 특징을 설명하시오."]

이처럼 모든 질문은 반드시 "~하시오."처럼 명령 어조이어야 합니다.
'''


In [ ]:
num_of_question = random.randint(3,5)
if num_of_question == 3 :
  format = '''
  "question 1"
  "Answer 1"
  "question 2"
  "Answer 2"
  "question 3"
  "Answer 3"'''
elif num_of_question == 4 :
  format = '''
  "question 1"
  "Answer 1"
  "question 2"
  "Answer 2"
  "question 3"
  "Answer 3"
  "question 4"
  "Answer 4"'''

else :
  format = '''
  "question 1"
  "Answer 1"
  "question 2"
  "Answer 2"
  "question 3"
  "Answer 3"
  "question 4"
  "Answer 4"
  "question 5"
  "Answer 5"'''

In [ ]:
test_text = f'''다음 제시문을 보고, 질문과 답변 세트를 {num_of_question}개 생성하라.

주의점은 다음과 같다.

1. 다른 불필요한 문장은 절대로 생성하지 마시오. 오직 질문과 답변만 생성하시오.
2. 질문은 1~2문장으로, 간결해야 한다.
3. 각 질문들은 서로 다른 내용으로, 겹치지 않아야 한다.
4. 모든 질문은 반드시 "~하시오."의 명령조 형식으로 끝나야 한다. 예를 들어 "제시문 [가]와 [나]에 기반하여 [다]에 대한 생각을 설명하시오."와 같은 형탱이어야 한다.
5. 또한 질문은 최대한 자연스러운 말투이어야 한다.
6. 지정한 질문과 답변의 세트 개수는, 제시문 전체에 대해서 적용되는 것이다.
7. 마지막으로 질문에는, 답변에 대한 내용이 절대로 나타나서는 안 된다. 즉, 질문은 구체적인 내용을 물어보는 것이 아닌 포괄적인 질문이어야 한다.
###

생성 형식은
{format}
으로 해야 한다.
새로운 제시문은 다음과 같다.

###
{paragraph2}
###

이제 질문과 답변 세트를 생성하라.
                '''

In [ ]:
prompts2 = translator.translate_text(PROMPT2, target_lang="EN-US").text
instruction2 = translator.translate_text(test_text, target_lang="EN-US").text

In [ ]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id, verbose = 0)

model = Llama(
    model_path='/content/Meta-Llama-3-8B-Instruct.Q4_1.gguf', #다운로드받은 모델의 위치
    n_ctx=5000,
    n_gpu_layers=-1, verbose = 0        # Number of model layers to offload to GPU
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


ValueError: Failed to load model from file: /content/Meta-Llama-3-8B-Instruct.Q4_1.gguf

In [ ]:
messages = [
  {"role": "system", "content": f"{prompts2}"},
  {"role": "user", "content": f"{instruction2}"}
  ]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True, tokenize = False,verbose = 0
)
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model(
    input_ids,
    max_tokens = 2000,
    temperature=0.6,
    top_p=0.8,
)

generated = outputs['choices'][0]['text'][len(outputs):]
print(generated)

TypeError: BertModel.forward() got an unexpected keyword argument 'max_tokens'

In [ ]:
import re

In [ ]:
# 정규 표현식을 사용하여 질문을 추출하는 함수
def extract_questions(text):
    questions = re.findall(r'\*\*Question \d+\*\*\n(.*?)\n', text, re.DOTALL)
    return questions

# 질문 추출
questions = extract_questions(generated)

# 추출된 질문 출력
for question in questions:
    print(question)

What is the fundamental nature of fashion, according to [a]?
How do the principles of the age of custom and the age of fashion differ from each other?
What drives the spread of fashions throughout society, according to [b]?
How do social roles influence language use, as described in [c]?
What is an example of a generational phenomenon in language use, according to [c]?


In [ ]:
questions

['What is the fundamental nature of fashion, according to [a]?',
 'How do the principles of the age of custom and the age of fashion differ from each other?',
 'What drives the spread of fashions throughout society, according to [b]?',
 'How do social roles influence language use, as described in [c]?',
 'What is an example of a generational phenomenon in language use, according to [c]?']

In [ ]:
q_list = []

In [ ]:
for i in questions :
  question = translator.translate_text(i, target_lang="KO").text
  if '패션' in question :
    question = question.replace('패션','유행')
  print(question)
  q_list.append(question)

A]에 따르면 유행의 근본적인 본질은 무엇인가요?
관습의 시대와 유행의 시대의 원칙은 서로 어떻게 다른가요?
B]에 따르면 사회 전반에 걸쳐 유행이 확산되는 원동력은 무엇인가요?
다]에 설명된 것처럼 사회적 역할이 언어 사용에 어떤 영향을 미치나요?
다]에 따르면 언어 사용의 세대적 현상의 예는 무엇인가요?


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto", n_gpu_layers=-1
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
messages = [
  {"role": "system", "content": f"{prompts}"},
  {"role": "user", "content": f"{instruction}"}
  ]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",verbose = 0
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=2000,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.8,
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Here are five question-answer sets based on the given paragraphs:

**Set 1**
"Describe the relationship between the desire to imitate and the need for individuality in the context of fashion."
The desire to imitate and the need for individuality are two fundamental forces that shape the world of fashion. Fashion is a unique phenomenon that satisfies both the desire to conform and the need to express oneself.

**Set 2**
"How do the principles of the age of fashion differ from those of the age of custom?"
The age of fashion is characterized by a tendency to worship the new and imitate contemporaries, whereas the age of custom is dominated by a tendency to follow the old or forefathers. The love of change and imitation of contemporaries drive the age of fashion, whereas reverence for legacy and tradition define the age of custom.

**Set 3**
"What is the role of social behavior in shaping language, as described in paragraph (c)?"
Social behavior plays a significant role in shaping language

In [ ]:
answer = tokenizer.decode(response, skip_special_tokens=True)
answer

'Here are five question-answer sets based on the given paragraphs:\n\n**Set 1**\n"Describe the relationship between the desire to imitate and the need for individuality in the context of fashion."\nThe desire to imitate and the need for individuality are two fundamental forces that shape the world of fashion. Fashion is a unique phenomenon that satisfies both the desire to conform and the need to express oneself.\n\n**Set 2**\n"How do the principles of the age of fashion differ from those of the age of custom?"\nThe age of fashion is characterized by a tendency to worship the new and imitate contemporaries, whereas the age of custom is dominated by a tendency to follow the old or forefathers. The love of change and imitation of contemporaries drive the age of fashion, whereas reverence for legacy and tradition define the age of custom.\n\n**Set 3**\n"What is the role of social behavior in shaping language, as described in paragraph (c)?"\nSocial behavior plays a significant role in sha

In [ ]:
import re

In [ ]:
questions = re.findall(r'"(.*?)"', answer)

# 결과 출력
for i, question in enumerate(questions, 1):
    print(f"Question {i}: {question}")

Question 1: Describe the relationship between the desire to imitate and the need for individuality in the context of fashion.
Question 2: How do the principles of the age of fashion differ from those of the age of custom?
Question 3: What is the role of social behavior in shaping language, as described in paragraph (c)?
Question 4: How do the concepts of fashion and language fads relate to each other?
Question 5: Explain how the concept of fashion as described in paragraph (a) relates to the idea of individuality.


In [ ]:
questions

['Describe the relationship between the desire to imitate and the need for individuality in the context of fashion.',
 'How do the principles of the age of fashion differ from those of the age of custom?',
 'What is the role of social behavior in shaping language, as described in paragraph (c)?',
 'How do the concepts of fashion and language fads relate to each other?',
 'Explain how the concept of fashion as described in paragraph (a) relates to the idea of individuality.']

In [ ]:
for i in questions :
  question = translator.translate_text(i, target_lang="KO").text
  if '패션' in question :
    question = question.replace('패션','유행')
  print(question)

유행의 맥락에서 모방 욕구와 개성의 필요성 사이의 관계를 설명하세요.
유행의 시대의 원칙은 관습의 시대와 어떻게 다를까요?
(다)항에 설명된 대로 언어를 형성하는 데 있어 사회적 행동의 역할은 무엇인가요?
유행과 언어 유행의 개념은 서로 어떤 관련이 있을까요?
단락 (가)에 설명된 유행의 개념이 개성이라는 개념과 어떤 관련이 있는지 설명하세요.
